macro
- macro variables
- modular
- parameters
- conditional logic
- call symput


In [1]:
import saspy
sas = saspy.SASsession()

Using SAS Config named: oda
SAS Connection established. Subprocess id is 18252



## macro


### macro variables


In [2]:
sas.submitLST(f"""
    %let flowertype = Ginger;
              
    data flowersales;
        infile datalines dlm=',' dsd;
        input customerid $ saledate :mmddyy10. variety :$ salequantity saleamount;
        datalines;
240W, 02-07-2012, Ginger, 120, 960
240W, 02-10-2012, Protea, 180, 1710
356W, 02-10-2012, Heliconia, 60, 720
356W, 02-15-2012, Anthurium, 300, 1050
188R, 02-16-2012, Ginger, 24, 192
188R, 02-18-2012, Anthurium, 24, 96
240W, 02-21-2012, Heliconia, 48, 600
240W, 02-27-2012, Protea, 48, 456
356W, 02-29-2012, Ginger, 240, 1980
;
    run;
              
    data flowersales_sub;
        set flowersales;
        if variety = "&flowertype";
    run;

    proc print data=flowersales_sub;
    format saledate worddate. saleamount dollar8.2;
    title "Sales of &flowertype..table";
    run;
""")

Obs,customerid,saledate,variety,salequantity,saleamount
1,240W,"February 7, 2012",Ginger,120,$960.00
2,188R,"February 16, 2012",Ginger,24,$192.00
3,356W,"February 29, 2012",Ginger,240,$1980.00


In [3]:
sas.submitLST(f"""
    %let sumvar = amount;
              
    proc means data=flowersales sum mean max min maxdec=2;
        var sale&sumvar;
        class variety;
        title "Summary of sales&sumvar by variety";
    run;
""")

### modular


In [4]:
sas.submitLST(f"""
    %macro sample;
        proc sort data = flowersales;
            by descending salequantity;
        run;
              
        proc print data = flowersales (obs = 5);
            format saledate worddate. saleamount dollar.;
            title 'five largest sales by quantity';
        run;
    %mend sample;

    %sample;
""")

Obs,customerid,saledate,variety,salequantity,saleamount
1,356W,"February 15, 2012",Anthuriu,300,"$1,050"
2,356W,"February 29, 2012",Ginger,240,"$1,980"
3,240W,"February 10, 2012",Protea,180,"$1,710"
4,240W,"February 7, 2012",Ginger,120,$960
5,356W,"February 10, 2012",Heliconi,60,$720


### parameters


In [5]:
sas.submitLST(f"""
    %macro select(customer=,sortvar=); 
        proc sort data = flowersales out = df; 
            by &sortvar; 
            where customerid = "&customer"; 
        run; 
              
        proc print data = df; 
            format saledate worddate. saleamount dollar.;
            title "orders for customer number &customer sorted by &sortvar"; 
        run; 
    %mend select; 
    
    %select(customer=240W,sortvar=saledate);
    %select(customer=240W,sortvar=saleamount);
""")

Obs,customerid,saledate,variety,salequantity,saleamount
1,240W,"February 7, 2012",Ginger,120,$960
2,240W,"February 10, 2012",Protea,180,"$1,710"
3,240W,"February 21, 2012",Heliconi,48,$600
4,240W,"February 27, 2012",Protea,48,$456
Obs,customerid,saledate,variety,salequantity,saleamount
1,240W,"February 27, 2012",Protea,48,$456
2,240W,"February 21, 2012",Heliconi,48,$600
3,240W,"February 7, 2012",Ginger,120,$960
4,240W,"February 10, 2012",Protea,180,"$1,710"


### conditional logic


### call symput


### others

In [6]:
sas.submitLST(f"""
/* head */
/* ######################################## */
%macro head(table_name);
   proc sql outobs=5;
      select * from &table_name.;
   quit;
%mend;

%head(sashelp.cars);

/* to avoid fetching the whole table */
%macro head(table_name);
    proc sql;
        connect to mysql (/* your connection options here */);
        select * from connection to mysql
        (
            select * from &table_name. limit 5
        );
        disconnect from mysql;
    quit;
%mend;

%head(my_table);

/* data step within a macro */
/* ######################################## */

%macro create_data(n);
    data work.sample;
        do i = 1 to &n;
            output;
        end;
    run;
%mend;

%create_data(5);

/* within loop within data step */
%macro create_datasets(n);
    %do i = 1 %to &n;
        data work.sample_&i;
            do j = 1 to &i;
                output;
            end;
        run;
    %end;
%mend;

%create_datasets(5);


/* logging */
/* ######################################## */

/* printing a word that shows up in the results page*/
DATA _NULL_;
    PUT '================= Section 1 Results =================';
RUN;

TITLE '================= Section 1 Results =================';
/* Your code for Section 1 here */

/* loops */
/* ######################################## */

/* a do loop that prints out 1 to 100 */
DATA _NULL_;
    DO I = 1 TO 100;
        PUT I;
    END;
RUN;

/* same but inside a macro */
%macro print_1_to_100;
    %do i = 1 %to 100;
        %put &i;
        /* print like these: on loop 3 */
        %put on loop &i;
    %end;
%mend;

%print_1_to_100;


/* variable */
/* ######################################## */

/* creating a global variable */
%let global_var = 5;
%put &global_var;

/* creating a local variable */
%macro local_var;
    %let local_var = 10;
    %put &local_var;
%mend;

%local_var;

/* creating a macro variable */

%macro create_macro_var;
    %let macro_var = 15;
    %put &macro_var;

    %global macro_var;
%mend;

%create_macro_var;

/* creating a macro variable with a value from a dataset */
DATA _NULL_;
    SET sashelp.class;
    CALL SYMPUT('class_size', _N_);
RUN;

%put &class_size;



run;
""")

c:\Users\yzdom\AppData\Local\Programs\Python\Python312\Lib\site-packages\saspy\sasioiom.py:1056: UserWarning: Noticed 'ERROR:' in LOG, you ought to take a look and see if there was a problem
  warnings.warn("Noticed 'ERROR:' in LOG, you ought to take a look and see if there was a problem")


Make,Model,Type,Origin,DriveTrain,MSRP,Invoice,Engine Size (L),Cylinders,Horsepower,MPG (City),MPG (Highway),Weight (LBS),Wheelbase (IN),Length (IN)
Acura,MDX,SUV,Asia,All,"$36,945","$33,337",3.5,6,265,17,23,4451,106,189
Acura,RSX Type S 2dr,Sedan,Asia,Front,"$23,820","$21,761",2,4,200,24,31,2778,101,172
Acura,TSX 4dr,Sedan,Asia,Front,"$26,990","$24,647",2.4,4,200,22,29,3230,105,183
Acura,TL 4dr,Sedan,Asia,Front,"$33,195","$30,299",3.2,6,270,20,28,3575,108,186
Acura,3.5 RL 4dr,Sedan,Asia,Front,"$43,755","$39,014",3.5,6,225,18,24,3880,115,197
